In [1]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
from __future__ import print_function, division
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features
import datetime

from tqdm import tqdm
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
from sklearn.metrics import mean_squared_error
import glob
from sklearn.decomposition import PCA
from itertools import combinations

os.listdir('../input/')
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_avtive_month']

### Data Load

In [24]:
#  Data Load
base = utils.read_pkl_gzip('../input/base_type_group.gz')
win_path = f'../features/4_winner/*.gz'
# win_path = f'../features/3_third_valid/*.gz'
model_path = '../model/LB3667_single_70leaves/*.gz'
win_path_list = glob.glob(win_path) + glob.glob(model_path)
# tmp_path_list = glob.glob('../features/5_tmp/*.gz')
# tmp_path_list = []
# win_path_list += tmp_path_list

# PCAにかけるfeature 
feim = pd.read_csv('../valid/0224_215_valid_lgb_lr0.01_272feats_10seed_70leaves_iter1161_OUT0_CV3.6176129805843_LB.csv')
top70 = feim['feature'].values[:100]

path_list = []
for feature in top70:
    if feature.count('PCA'):continue
    if feature[:3]=='203':continue
    if feature[:3]=='556':continue
    for path in win_path_list:
        if path.count('PCA'):continue
        if path.count('556'):continue
        if path.count(feature):
            path_list.append(path)

base_train = base[~base[target].isnull()].reset_index(drop=True)
base_test = base[base[target].isnull()].reset_index(drop=True)
feature_list = utils.parallel_load_data(path_list=path_list)
df = pd.concat(feature_list, axis=1)

In [26]:
for col in df.columns:
    df[col] = utils.impute_feature(df=df, col=col)
    
print(df.shape)
df.head()

(325540, 40)


,206_cop_diff_auth1_cat1_lag05_auth1_cat0_lag05_purchase_amount_min@,214_pst_ratio_auth1_cat1_lag0_auth1_cat0_lag0_CLV@,206_pst_auth1_cat1_lag05_diff_date_from_hist_min_month_min@,206_pst_auth1_cat0_lag05_monthly_avg_diff_date_from_hist_max_month_min@,206_pst_auth1_cat0_month_diff_var@,206_pst_new_cat0_diff_date_from_hist_max_month_min@,206_pst_auth1_cat0_lag02_purchase_amount_var@,206_pst_auth1_cat0_diff_date_from_hist_max_month_max@,200_ker_hist_duration_min@,206_pst_auth1_cat0_monthly_avg_diff_date_from_hist_max_month_std@,206_pst_auth1_cat1_diff_date_from_hist_min_month_min@,206_pst_auth1_cat0_lag05_monthly_avg_diff_date_from_hist_max_month_max@,206_cop_ratio_new_cat0_auth1_cat0_diff_date_from_hist_max_month_mean@,206_cop_ratio_auth1_cat0_lag02_auth1_cat0_lag05_purchase_amount_mean@,130_ddf_all_hist_last25_date_diff_std@,206_pst_auth1_cat1_lag1_diff_date_from_hist_max_month_mean@,206_pst_auth1_cat1_monthly_avg_diff_date_from_hist_max_month_std@,206_pst_auth1_cat1_lag05_diff_date_from_hist_max_month_max@,206_pst_auth1_cat1_monthly_avg_purchase_amount_min@,152_ah1__auth1_hist_month_nunique@,214_pst_diff_new_cat0_new_cat0_lag1_CLV@,267_ath_lastmonth3_authorized_flag_mean@,214_pst_auth1_cat1_duration_std@,215_ins_rem_ins_cnt_mean@,152_ah0__auth0_hist_purchase_amount_min@,206_pst_auth1_cat1_monthly_avg_purchase_amount_var@,111_ker_new_purchase_date_uptonow@,206_pst_auth1_cat0_monthly_avg_month_diff_mean@,206_pst_auth1_cat0_month_diff_mean@,200_ker_hist_CLV@,206_pst_new_cat0_lag2_diff_date_from_new_max_month_mean@,206_pst_new_cat0_monthly_avg_diff_date_from_hist_max_month_mean@,245_rad_min_Mlag-6_auth1_all_top-M2_merchant_flag0_-2-plag-3_-5_purchase_amount_new_sum_per_installments_sum@,241_rad_auth1_all_flag-1_-3-plag-4_-9_purchase_amount_new_sum_per_installments_sum_mean@,206_cop_ratio_auth1_cat1_lag05_auth1_cat0_lag05_month_diff_mean@,152_ah0__auth0_hist_month_lag_var@,115_out_state_id_-1@,206_pst_auth1_cat1_month_diff_mean@,129_tzr_all_timezone_ratio_afternoon@,190_fe1_all_auth1_amount_per_installments_sum@
0,-2.999238,-0.999999,-425.0,75.833336,0.213884,-60.0,0.056793,246.0,-8.804688,7.607743,-425.0,102.833336,-0.237327,1.141544,7.712111,-1.0,-1.000000,-1.0,-0.999975,9.0,17.999023,0.960452,-1.000000,0.000000,0.008916,-1.000000,278.0,11.267158,11.307693,-3917.187012,15.636364,-31.848484,1.269764,1.342060,0.090909,2.769231,0.000000,23.000000,0.247350,26.093750
1,-0.001503,0.000804,-372.0,77.500000,0.216891,-58.0,0.007603,390.0,-9.593750,8.245891,-372.0,104.666664,-0.172240,0.643320,3.426855,45.5,6.591889,136.0,0.312744,12.0,0.270905,0.973016,6.497865,7.411765,0.005911,0.205322,308.0,12.331050,12.316129,-6217.010254,7.666667,-27.500000,8.735700,0.963889,0.998872,7.272727,0.087079,12.103448,0.264045,0.081238
2,-2.999238,-0.999999,-425.0,83.599998,0.189024,-59.0,0.073181,413.0,-8.726562,6.421233,-425.0,88.400002,-0.219909,1.760263,99.605019,-1.0,-1.000000,-1.0,-0.999975,10.0,-331.531250,1.000000,-1.000000,0.000000,0.045355,-1.000000,278.0,11.318055,11.243902,-113.311218,2.000000,-59.000000,-1.000000,0.745148,0.090909,2.000000,0.000000,23.000000,0.318182,2.794922
3,-0.001488,0.942190,-174.0,76.166664,0.196154,-48.0,0.005726,155.0,-8.843750,7.747190,-144.0,96.333336,-0.317355,1.142039,22.115831,-1.0,8.102212,141.0,0.109985,6.0,1.048340,1.000000,5.936644,1.600000,-1.000100,0.269531,289.0,11.276471,11.261539,-350.128998,19.250000,-28.625000,0.212190,0.579901,1.013775,-1.000000,0.154762,11.083333,0.464286,0.089966
4,-0.008264,0.045748,-120.0,45.250000,0.197526,-59.0,0.029434,109.0,-8.953125,8.321506,-120.0,70.750000,-0.624641,1.017544,6.397135,46.0,8.781362,96.0,0.423828,4.0,75.724609,0.977848,8.064681,1.076923,0.233863,0.472412,278.0,11.239293,11.267241,-905.339905,16.105263,-30.380701,2.701793,0.000000,1.020658,0.300000,0.100592,11.416667,0.195266,0.137207


In [27]:
#========================================================================
# Regular Featureに対するPCA 
prefix = '222'
prefix = '322'
prefix = '270'

pca = PCA(n_components=0.81,whiten=True)

combi_list = combinations(df.columns, 2)

pca_list = []
for combi in combi_list:
    tmp =  df[list(combi)]
    feature = pca.fit_transform(tmp)
    if feature.ravel().shape[0]>500000:
        feat_1 = feature[:, 0]
        feat_2 = feature[:, 1]
        utils.to_pkl_gzip(obj=feat_1, path=f'../features/1_first_valid/{prefix}_PCA_D1_{str(combi)}')
        utils.to_pkl_gzip(obj=feat_2, path=f'../features/1_first_valid/{prefix}_PCA_D2_{str(combi)}')
    else:
        feature = feature.ravel().astype('float32')
        utils.to_pkl_gzip(obj=feature, path=f'../features/1_first_valid/{prefix}_PCA_{str(combi)}')
#========================================================================

In [23]:
#========================================================================
# Regular Featureからは独立していそうなFeatureに対するPCA.
# もしくは、強力だがぶつかってしまうfeatureについて軸を変えてみる
prefix = '223'

pca = PCA(n_components=0.81,whiten=True)

path_list = glob.glob('../features/2_second_valid/558*.gz')
feature_list = utils.parallel_load_data(path_list=path_list)
df = pd.concat(feature_list, axis=1)
combi_list = combinations(df.columns, 2)


for combi in combi_list:
    col0 = combi[0]
    col1 = combi[1]
    feat0 = df[col0]
    feat1 = df[col1]
    if col0.count('date') or col0.count('month_diff'):
        if feat0.min()>=0:
            feat0.fillna(-1, inplace=True)
        else:
            feat0.fillna(feat0.min()-10, inplace=True)
    else:
        feat0.fillna(-1, inplace=True)
        
    if col1.count('date') or col1.count('month_diff'):
        if feat1.min()>=0:
            feat1.fillna(-1, inplace=True)
        else:
            feat1.fillna(feat1.min()-10, inplace=True)
    else:
        feat1.fillna(-1, inplace=True)
        
pca_list = []
impute_list = []
for combi in combi_list:
    try:
        feature = pca.fit_transform(df[list(combi)])
    except ValueError:
        impute_list += list(combi)
        continue
    if feature.ravel().shape[0]>500000:
        feat_1 = feature[:, 0]
        feat_2 = feature[:, 1]
        utils.to_pkl_gzip(obj=feat_1, path=f'../features/1_first_valid/{prefix}_PCA_D1_{str(combi)}')
        utils.to_pkl_gzip(obj=feat_2, path=f'../features/1_first_valid/{prefix}_PCA_D2_{str(combi)}')
    else:
        feature = feature.ravel().astype('float32')
        utils.to_pkl_gzip(obj=feature, path=f'../features/1_first_valid/{prefix}_PCA_{str(combi)}')
#========================================================================

In [14]:
path_list = glob.glob('../features/5_tmp/558*.gz')

In [24]:
for path in impute_list:
    print(path)
#     if utils.read_pkl_gzip(path).shape[0]>330000:
#         print(path)